In [240]:
with open("input18.txt") as f:
    inp = [x[:-1] for x in f.readlines()]
inp

['[[[[6,3],7],0],[[7,0],0]]',
 '[[[4,7],[6,[6,5]]],[4,[[6,5],[9,1]]]]',
 '[[[[3,7],[6,9]],[3,[4,1]]],8]',
 '[[[0,[2,0]],3],2]',
 '[[[[1,3],4],9],[[[1,8],[9,3]],[0,7]]]',
 '[[[5,9],1],[[[4,8],[4,8]],[[9,7],[3,6]]]]',
 '[[[0,7],4],[[[0,4],2],[4,2]]]',
 '[[5,1],[2,5]]',
 '[[[[4,8],[8,3]],[6,[2,3]]],[[6,0],[1,3]]]',
 '[[[[1,7],[8,1]],[2,[3,4]]],[[7,[8,7]],[[8,6],5]]]',
 '[[9,[[8,9],[0,6]]],[[[8,4],5],[0,[1,7]]]]',
 '[[[[9,8],[6,9]],[[3,5],[6,2]]],[[[7,8],5],8]]',
 '[[[[7,1],8],[0,2]],[3,5]]',
 '[[[1,9],3],[8,[8,[7,8]]]]',
 '[[[9,[8,5]],[1,[9,0]]],5]',
 '[9,[[[1,1],8],[[3,5],9]]]',
 '[[[1,[7,8]],2],[8,0]]',
 '[7,[8,[[6,1],[7,9]]]]',
 '[[7,[[4,7],7]],[[8,[5,2]],2]]',
 '[[[[1,7],[9,0]],3],[8,[4,[2,0]]]]',
 '[[4,3],[[9,[7,7]],7]]',
 '[[[[9,5],3],[[8,5],5]],5]',
 '[[[[4,9],2],[[5,6],[9,0]]],[[2,[2,2]],1]]',
 '[[[[7,9],[6,0]],[1,[5,8]]],[8,8]]',
 '[[[[0,5],2],[4,[5,7]]],[8,[[8,1],[6,7]]]]',
 '[9,[7,[[3,7],[6,4]]]]',
 '[[[[5,3],[5,2]],[[5,0],0]],[[[4,6],[6,4]],[8,8]]]',
 '[1,[[8,0],0]]',
 '[[3,[7

In [246]:
import math

class Node():
    def __init__(self, value, left, right):
        self.value = value
        self.left = left
        self.right = right

def read_tree(l):
    if type(l) is int:
        return Node(l, None, None)
    else:
        return Node(None, read_tree(l[0]), read_tree(l[1]))

def find_leftmost(tree, cond):
    """Returns path to the leftmost leaf satisfying cond or False"""
    path = ""
    if tree.value != None:
        return (True,"") if cond(tree.value) else (False, "")
    bol, path = find_leftmost(tree.left, cond)
    if bol:
        return (True, 'L' + path)
    bol, path = find_leftmost(tree.right, cond)
    if bol:
        return (True, 'R' + path)    
    return (False, path)

def access(tree, path):
    for i in path:
        tree = tree.left if i=='L' else tree.right
    return tree

def modify(tree, path, new_value):
    """Modifies a tree node by making it a leaf with new_value"""
    if not path:
        return read_tree(new_value)
    return Node(None, modify(tree.left, path[1:], new_value), tree.right) if path[0]=='L' \
                else Node(None, tree.left, modify(tree.right, path[1:], new_value))

def represent(tree):
    if tree.value != None:
        return tree.value
    return [represent(tree.left), represent(tree.right)]

def left(tree, path):
    """Given a path to a leaf, returns path to left leaf"""
    if 'R' not in path:
        return False
    lastR  = [i for i,x in enumerate(path) if x == 'R'][-1]
    new_path = path[:lastR] + 'L'
    for i in new_path:
        tree = tree.left if i=='L' else tree.right
    while tree.value == None:
        new_path += 'R'
        tree = tree.right
    return new_path

def right(tree, path):
    """Given a path to a leaf, returns path to left leaf"""
    if 'L' not in path:
        return False
    lastL  = [i for i,x in enumerate(path) if x == 'L'][-1]
    new_path = path[:lastL] + 'R'
    for i in new_path:
        tree = tree.left if i=='L' else tree.right
    while tree.value == None:
        new_path += 'L'
        tree = tree.left
    return new_path

def leftmost_depth(tree, n):
    """Returns path with leftmost leaf with depth n"""
    if tree.value != None: 
        return (True, "") if n==0 else (False, "")
    bol, path = leftmost_depth(tree.left, n-1)
    if bol:
        return (True, 'L' + path)
    bol, path = leftmost_depth(tree.right, n-1)
    if bol:
        return (True, 'R' + path)
    return (False, "")
    
def explode(tree):
    bol, path = leftmost_depth(tree, 5)
    if not bol: return (False, tree)
    pair = access(tree, path[:-1])
    leftnum = pair.left.value
    rightnum = pair.right.value
    leftnode = left(tree, path)
    rightnode = right(tree, path[:-1]+'R')
    tree = modify(tree, path[:-1], 0)
    if leftnode:
        tree = modify(tree, leftnode, access(tree, leftnode).value + leftnum)
    if rightnode:
        tree = modify(tree, rightnode, access(tree, rightnode).value + rightnum)
    return (True, tree)
    

def split(tree):
    bol, path = find_leftmost(tree, lambda x: x>9)
    if not bol: return (False, tree)
    num = access(tree, path).value
    tree = modify(tree, path, [math.floor(num/2), math.ceil(num/2)])
    return (True, tree)

def addition(t1, t2):
    tree = Node(None, t1, t2)
    end = False
    while not end:
        bol, exp = explode(tree)
        bol2, spl = split(tree)
        if bol:
            tree = exp
        elif not bol and bol2:
            tree = spl
        else:
            end = True
    return tree
            
def solve(inp):
    tree = read_tree(eval(inp[0]))
    for t in inp[1:]:
        tree = addition(tree, read_tree(eval(t)))
    return tree

def magnitude(tree):
    if tree.value != None:
        return tree.value
    return 3*magnitude(tree.left) + 2*magnitude(tree.right)

magnitude(solve(inp))

4457

# Part 2

In [251]:
def part2(inp):
    res = 0
    for i in inp:
        for j in inp:
            if i != j:
                partial = magnitude(addition(read_tree(eval(i)), read_tree(eval(j))))
                if partial > res:
                    res = partial
    return res

part2(inp)

4784